# Beyond sequences primary structure


We may have access to sequences 3D-structure (which could be predicted by a tool). How can we use seqme to evaluate sequences based on their 3D-structure? We will show how to do this in this notebook.


In [ ]:
# !pip install BioPython

In [ ]:
from typing import Literal

import numpy as np
from Bio.Align import PairwiseAligner

import seqme as sm

In [ ]:
# IMPORTANT! Don't use this to compute RMSD (we use it to stay BSD 3-clause compatible). There are better ways to align proteins, e.g., TM-align.


def transform(coords1: np.ndarray, coords2: np.ndarray, *, with_scale: bool = True, return_params: bool = False):
    """Align coords2 to coords1 (both arrays shape (N,3) or (N,d)).

    Minimizes least-squares error for rotation, translation and optional uniform scale.

    Args:
        coords1: target points, shape (N, d)
        coords2: source points to transform, shape (N, d)
        with_scale: if True, estimate uniform scale; if False, force scale=1
        return_params: if True, return (aligned_coords2, R, t, s); otherwise just aligned coords

    Returns:
        aligned_coords2 (and optionally) rotation matrix R (dxd), translation vector t (d,),
        scale s (float).
    """
    A = np.asarray(coords1, dtype=float)
    B = np.asarray(coords2, dtype=float)

    if A.shape != B.shape:
        raise ValueError("coords1 and coords2 must have the same shape")

    N, d = A.shape
    if N == 0:
        raise ValueError("need at least one point")

    # centroids
    mu_A = A.mean(axis=0)
    mu_B = B.mean(axis=0)

    AA = A - mu_A
    BB = B - mu_B

    # covariance / cross-covariance
    cov = (AA.T @ BB) / N

    # SVD
    U, D, Vt = np.linalg.svd(cov)
    V = Vt.T

    # handle reflection
    S = np.eye(d)
    if np.linalg.det(U) * np.linalg.det(V) < 0:
        S[-1, -1] = -1

    R = U @ S @ V.T

    if with_scale:
        var_B = (BB * BB).sum() / N
        # trace(D * S) is sum(D * diag(S))
        s = float(np.sum(D * np.diag(S)) / var_B)
    else:
        s = 1.0

    t = mu_A - s * (R @ mu_B)

    transformed = (s * (R @ B.T)).T + t

    if return_params:
        return transformed, R, t, s
    return transformed


def rmsd(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a)
    b = np.asarray(b)
    return float(np.sqrt(((a - b) ** 2).sum() / a.shape[0]))


def indices(s1: str, s2: str) -> np.ndarray:
    res = []
    i = 0
    for c1, c2 in zip(s1, s2, strict=True):
        if c1 != "-":
            if c2 != "-":
                res.append(i)
            i += 1
    return np.array(res)


def compute_rmsd(coords1: np.ndarray, coords2: np.ndarray, seq1: str, seq2: str) -> float:
    align = PairwiseAligner().align(seq1, seq2)[0]
    a_seq1, a_seq2 = align[0], align[1]
    coords1 = coords1[indices(a_seq1, a_seq2)]
    coords2 = coords2[indices(a_seq2, a_seq1)]

    coords2_aligned = transform(coords1, coords2)
    return rmsd(coords1, coords2_aligned)

Let's define a metric which uses atomic positions. Here we use RMSD.


In [ ]:
class RMSD(sm.Metric):
    """Root mean square deviation of atomic positions."""

    def __init__(self, reference: str, sequence_to_coordinates: dict[str, np.ndarray]):
        self.reference = reference
        self.sequence_to_coordinates = sequence_to_coordinates

    def __call__(self, sequences: list[str]) -> sm.MetricResult:
        ref_coords = self.sequence_to_coordinates[self.reference]
        scores = np.array(
            [compute_rmsd(self.sequence_to_coordinates[seq], ref_coords, seq, self.reference) for seq in sequences]
        )
        return sm.MetricResult(scores.mean().item())

    @property
    def name(self) -> str:
        return "RMSD"

    @property
    def objective(self) -> Literal["minimize", "maximize"]:
        return "minimize"

Let's define our protein folding model.

In [ ]:
cache = sm.ModelCache(models={"esm-fold": sm.models.EsmFold()})

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Note, we will only extract the position of the amino-acids "Cα" from the fold prediction.

In [ ]:
sequences = [
    "MRKIVVAAIAVSLTTVSITASASADPSKDSKAQVSAAEAGITGTWYNQLGSTFIVTAGADGALTGTYESAVGNAESRYVLTGRYDSAPATDGSGTALGWTVAWKNNYRNAHSATTWSGQYVGGAEARINTQWLLTSGTTEANAWKSTLVGHDTFTKVKPSAASIDAAKKAGVNNGNPLDAVQQ",
    "MVHATSPLLLLLLLSLALVAPGLSARKCSLTGKWTNDLGSNMTIGAVNSRGEFTGTYITAVTATSNEIKESPLHGTQNTINKRTQPTFGFTVNWKFSESTTVFTGQCFIDRNGKEVLKTMWLLRSSVNDIGDDWKATRVGINIFTRLRTQKE",
]

# sequences = ["CC", "RRR"]
coordinates = cache.model("esm-fold", variable_length=True)(sequences)
sequence_to_coordinates = dict(zip(sequences, coordinates, strict=True))

Let's create the metric and sequences.


In [ ]:
metrics = [RMSD(reference=sequences[0], sequence_to_coordinates=sequence_to_coordinates)]

In [ ]:
sequences = {f"Protein {i + 1}": [seq] for i, seq in enumerate(list(sequence_to_coordinates.keys()))}

Let's compute the metric.


In [ ]:
df = sm.evaluate(sequences, metrics)

100%|██████████| 2/2 [00:00<00:00, 47.88it/s, data=Protein 2, metric=RMSD]


In [ ]:
sm.show(df)

,RMSD↓
Protein 1,0.00
Protein 2,15.98


Recall seqme defines three groups of metrics: sequence-based, embedding-based and property-based metrics. One may ask, what group this metric fits in? Notice, metrics operating on 3D-structure are very similar to property-based metrics: sequence → 3D-structure (property) → metric. And there you go.